In [1]:
print("OK")

OK


In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain


In [3]:
%pwd

'/Users/sseadmin/Documents/personal_repos/Gen-ai-learn/projects/code-analysis/research'

In [8]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI", to_path=repo_path)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git clone -v -- https://github.com/entbappy/End-to-end-Medical-Chatbot-Generative-AI test_repo/
  stderr: 'fatal: destination path 'test_repo' already exists and is not an empty directory.
'

In [6]:
loader = GenericLoader.from_filesystem(repo_path,
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [7]:
documents = loader.load()

In [9]:
documents[0]

Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Bappy Ahmed',\n    author_email= 'entbappy73@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)")

In [10]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 500,
                                                             chunk_overlap = 20)

In [11]:
texts = documents_splitter.split_documents(documents)
texts

[Document(metadata={'source': 'test_repo/setup.py', 'language': <Language.PYTHON: 'python'>}, page_content="from setuptools import find_packages, setup\n\nsetup(\n    name = 'Generative AI Project',\n    version= '0.0.0',\n    author= 'Bappy Ahmed',\n    author_email= 'entbappy73@gmail.com',\n    packages= find_packages(),\n    install_requires = []\n\n)"),
 Document(metadata={'source': 'test_repo/app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_openai import OpenAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\nload_dotenv()'),
 Document(metadata={'source': 

In [12]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

In [17]:
embeddings = OllamaEmbeddings(model="llama3.2",base_url="http://localhost:11434");


In [18]:
sample_embedding = embeddings.embed_query("test query")
print(len(sample_embedding)) 

3072


In [19]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./new_db')

In [20]:
llm=Ollama(model="qwen2.5-coder:1.5b",base_url="http://localhost:11434");

/var/folders/p1/2pgfb5b50c343r108b91w7f80000gn/T/ipykernel_84247/212341753.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="qwen2.5-coder:1.5b",base_url="http://localhost:11434");


In [21]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

/var/folders/p1/2pgfb5b50c343r108b91w7f80000gn/T/ipykernel_84247/2101274949.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [22]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":8}), memory=memory)

In [23]:
question = "what is download_hugging_face_embeddings funtion?"

In [24]:
result = qa(question)
print(result['answer'])

/var/folders/p1/2pgfb5b50c343r108b91w7f80000gn/T/ipykernel_84247/79176006.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)
Number of requested results 20 is greater than number of elements in index 13, updating n_results = 13


The `download_hugging_face_embeddings` function downloads the pre-trained embeddings from Hugging Face using the `HuggingFaceEmbeddings` class from the LangChain library. This embedding model, specifically `'sentence-transformers/all-MiniLM-L6-v2'`, returns 384-dimensional vector representations of words and sentences.


In [25]:
question = "what is load_pdf_file funtion?"
result = qa(question)
print(result['answer'])

Number of requested results 20 is greater than number of elements in index 13, updating n_results = 13


The `load_pdf_file` function is responsible for extracting data from a specified PDF file and returning it as a list of documents. This is necessary because the subsequent steps in the code process these documents using the `langchain.document_loaders.PyPDFLoader`.
